## 对二噁英的原始数据进行清洗

从excel文件中读取二噁英的影响因子和结果，将其拼接到一个sheet里面。

In [1]:
#-*- coding :utf-8-*-
from functools import reduce

import pandas as pd

ereying='二噁英工控数据（1#炉）.xlsx'#二噁英数据

data1 = pd.read_excel(ereying, sheet_name=u'尾部烟道进口烟气温度')
data2 = pd.read_excel(ereying, sheet_name=u'炉尾部烟道出口烟气温度')
data3 = pd.read_excel(ereying, sheet_name=u'炉膛出口O2浓度1')
data4 = pd.read_excel(ereying, sheet_name=u'炉膛出口O2浓度2')
data5 = pd.read_excel(ereying, sheet_name=u'CO的浓度')
data6 = pd.read_excel(ereying, sheet_name=u'炉膛第一烟道出口烟气温度')
data7 = pd.read_excel(ereying, sheet_name=u'炉膛第一烟道入口烟气温度')
data8 = pd.read_excel(ereying, sheet_name=u'活性炭喷射量')
data9 = pd.read_excel(ereying, sheet_name=u'一次风机实际运行电流')
data10 = pd.read_excel(ereying, sheet_name=u'二次风机实际运行电流')
data11 = pd.read_excel(ereying, sheet_name=u'锅炉蒸汽量')
data12 = pd.read_excel(ereying, sheet_name=u'二噁英排放浓度')


dfs=[data1,data2,data3,data4,data5,data6,data7,data8,data9,data10,data11,data12]
df_final=reduce(lambda left,right:pd.merge(left,right,on='时间',how='outer'), dfs)
names = [u'时间',u'尾部烟道进口烟气温度',u'炉尾部烟道出口烟气温度',u'炉膛出口O2浓度1',u'炉膛出口O2浓度2',u'CO的浓度',u'炉膛第一烟道出口烟气温度',
         u'炉膛第一烟道入口烟气温度',u'活性炭喷射量',u'一次风机实际运行电流',u'二次风机实际运行电流',u'锅炉蒸汽量',u'二噁英排放浓度']
df_final.columns = names
df_final.to_excel('X_y.xlsx')

/home/yu/anaconda3/lib/python3.6/site-packages/pandas/util/_decorators.py:118: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


整理数据，每一个小时的测量值相加，按照时间进行重新排序，将结果转换成为csv文件,空白的地方写0

In [3]:
import pandas as pd
ereying=pd.DataFrame(pd.read_excel('X_y.xlsx'))
ereying=ereying.set_index(u'时间')

ereying2=ereying.resample('H').sum()

ereying2.to_csv('X_y.csv',encoding='utf-8')

由数据得知前面没有二噁英浓度的数据，删除前面部分

In [6]:
import pandas as pd

data = pd.DataFrame(pd.read_csv('X_y.csv'))
data.iloc[1549:, :].to_csv('X_y_12-04.csv', index=False)

对数据中二噁英浓度为0的行删除

In [3]:
import pandas as pd
data = pd.DataFrame(pd.read_csv('X_y_12-04.csv')
                   )
data[data[u'二噁英排放浓度']!=0].to_csv('X_y_nozero.csv', index=False)
print(data.shape)
# data.to_csv('X_y_nozero.csv')
data = pd.DataFrame(pd.read_csv('X_y_nozero.csv')
                   )
data[data[u'锅炉蒸汽量']!=0].to_csv('X_y_nozero1.csv', index=False)

(6443, 13)


将所有影响因子全为零但二噁英浓度不为0的行删除

In [4]:
import pandas as pd
data = pd.DataFrame(pd.read_csv('X_y_nozero1.csv')
                   )
data[data[u'尾部烟道进口烟气温度']!=0].to_csv('X_y_nozero2.csv', index=False)
data = pd.DataFrame(pd.read_csv('X_y_nozero2.csv')
                   )
data[data[u'炉膛出口O2浓度1']!=-30].to_csv('X_y_nozero3.csv', index=False)


训练数据和标签进行分离

In [5]:
import pandas as pd

data = pd.DataFrame(pd.read_csv('X_y_nozero3.csv'))

X_train = data.iloc[:, 1:12]

X_train.to_csv('X_train.csv', index=False)
Y_label = data.iloc[:,-1]
names = [u'二噁英排放浓度']
Y_label.columns = 'times'
Y_label.to_csv('Y_label.csv', index=False, header=u'二噁英排放浓度')

分开训练集和验证集

In [7]:
import pandas as pd
#训练集和验证集分离
data_train = pd.DataFrame(pd.read_csv('X_train.csv'))

X_train_split = data_train.iloc[400:4871, :]
X_val_split = data_train.iloc[0:400, :]

X_train_split.to_csv('X_train_split.csv', index=False)
X_val_split.to_csv('X_val_split.csv', index=False)

#标签分离
data_label = pd.DataFrame(pd.read_csv('Y_label.csv'))
print(len(data_label))
Y_train_split = data_label.iloc[400:4871, :]
Y_val_split = data_label.iloc[0:400, :]

Y_train_split.to_csv('Y_train_split.csv', index=False)
Y_val_split.to_csv('Y_val_split.csv', index=False)



4852


In [8]:
import pandas as pd

X_data = pd.DataFrame(pd.read_csv('X_val_split.csv'))
i= 50
for i in range(120):
    data1 = X_data.drop([i])
    X_data = data1
    print(data1.shape)
data1.to_csv('X_val_split_clear.csv', index=False)

Y_data = pd.DataFrame(pd.read_csv('Y_val_split.csv'))
i= 50
for i in range(120):
    data2 = Y_data.drop([i])
    Y_data = data2
    print(data2.shape)
data2.to_csv('Y_val_split_clear.csv', index=False)

(399, 11)
(398, 11)
(397, 11)
(396, 11)
(395, 11)
(394, 11)
(393, 11)
(392, 11)
(391, 11)
(390, 11)
(389, 11)
(388, 11)
(387, 11)
(386, 11)
(385, 11)
(384, 11)
(383, 11)
(382, 11)
(381, 11)
(380, 11)
(379, 11)
(378, 11)
(377, 11)
(376, 11)
(375, 11)
(374, 11)
(373, 11)
(372, 11)
(371, 11)
(370, 11)
(369, 11)
(368, 11)
(367, 11)
(366, 11)
(365, 11)
(364, 11)
(363, 11)
(362, 11)
(361, 11)
(360, 11)
(359, 11)
(358, 11)
(357, 11)
(356, 11)
(355, 11)
(354, 11)
(353, 11)
(352, 11)
(351, 11)
(350, 11)
(349, 11)
(348, 11)
(347, 11)
(346, 11)
(345, 11)
(344, 11)
(343, 11)
(342, 11)
(341, 11)
(340, 11)
(339, 11)
(338, 11)
(337, 11)
(336, 11)
(335, 11)
(334, 11)
(333, 11)
(332, 11)
(331, 11)
(330, 11)
(329, 11)
(328, 11)
(327, 11)
(326, 11)
(325, 11)
(324, 11)
(323, 11)
(322, 11)
(321, 11)
(320, 11)
(319, 11)
(318, 11)
(317, 11)
(316, 11)
(315, 11)
(314, 11)
(313, 11)
(312, 11)
(311, 11)
(310, 11)
(309, 11)
(308, 11)
(307, 11)
(306, 11)
(305, 11)
(304, 11)
(303, 11)
(302, 11)
(301, 11)
(300, 11)
